<a href="https://colab.research.google.com/github/RanjithS581/speech_emotion_recognition/blob/main/ranjith_Speech_Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Task - 4 : speech emotion recognition**

In [ ]:
import os
Root = "/content/drive/MyDrive/Colab_Notebooks/RAVDESS_Emotional_speech_audio"
os.chdir(Root)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ls

speech-emotion-recognition-ravdess-data/


In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result=np.hstack((result, mel))
    return result

In [ ]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/Colab_Notebooks/RAVDESS_Emotional_speech_audio/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
x_train

array([[-5.23220215e+02,  4.35375481e+01, -1.07963867e+01, ...,
         6.95891620e-04,  7.39991432e-04,  5.03907737e-04],
       [-5.14645447e+02,  4.42508736e+01, -4.14697933e+00, ...,
         5.80090971e-04,  3.33600270e-04,  2.12451690e-04],
       [-6.44363586e+02,  3.98517609e+01, -8.37395668e+00, ...,
         1.77935763e-05,  5.19435525e-05,  1.34935281e-05],
       ...,
       [-6.38889038e+02,  4.68636932e+01,  4.74644452e-03, ...,
         1.31693756e-04,  1.22964979e-04,  8.94539771e-05],
       [-5.53571289e+02,  3.47704163e+01, -2.40211439e+00, ...,
         6.17825834e-04,  3.78832250e-04,  1.81637995e-04],
       [-4.95604401e+02,  2.82074986e+01, -5.81012535e+00, ...,
         3.14757781e-04,  2.04955693e-04,  1.31822351e-04]])

In [ ]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [ ]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [ ]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
#Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
y_pred

array(['fearful', 'happy', 'fearful', 'calm', 'fearful', 'happy',
       'fearful', 'fearful', 'disgust', 'calm', 'fearful', 'fearful',
       'happy', 'disgust', 'disgust', 'fearful', 'calm', 'calm',
       'fearful', 'disgust', 'happy', 'disgust', 'fearful', 'fearful',
       'disgust', 'fearful', 'fearful', 'disgust', 'happy', 'calm',
       'disgust', 'disgust', 'fearful', 'happy', 'disgust', 'happy',
       'disgust', 'disgust', 'calm', 'calm', 'happy', 'disgust', 'happy',
       'calm', 'happy', 'calm', 'disgust', 'disgust', 'calm', 'happy',
       'fearful', 'fearful', 'fearful', 'fearful', 'calm', 'fearful',
       'fearful', 'calm', 'fearful', 'happy', 'fearful', 'calm',
       'fearful', 'disgust', 'disgust', 'calm', 'disgust', 'disgust',
       'fearful', 'fearful', 'fearful', 'disgust', 'happy', 'fearful',
       'fearful', 'fearful', 'disgust', 'happy', 'calm', 'fearful',
       'fearful', 'calm', 'fearful', 'calm', 'calm', 'fearful', 'happy',
       'happy', 'fearful', 'd

In [ ]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 65.10%


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
f1_score(y_test, y_pred,average=None)

array([0.71428571, 0.60869565, 0.62962963, 0.65116279])

In [ ]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,happy,fearful
1,happy,happy
2,happy,fearful
3,calm,calm
4,disgust,fearful
5,happy,happy
6,happy,fearful
7,calm,fearful
8,disgust,disgust
9,happy,calm


In [ ]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [ ]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/Colab_Notebooks/RAVDESS_Emotional_speech_audio/speech-emotion-recognition-ravdess-data/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['fearful'], dtype='<U7')

In [ ]:
feature

array([[-6.93497009e+02,  5.00643921e+01,  5.71450531e-01,
         1.43299656e+01,  3.33637071e+00, -2.54071975e+00,
        -4.05790949e+00, -1.07119999e+01, -7.29413939e+00,
         1.74018872e+00, -4.19064236e+00,  1.95466173e+00,
        -5.24789381e+00,  2.78142977e+00, -3.16756773e+00,
        -3.40008307e+00, -2.37803197e+00, -5.68717539e-01,
        -6.47753334e+00, -1.24320555e+00, -2.80542517e+00,
        -5.43635798e+00, -4.46875393e-01, -3.63516593e+00,
        -2.98372602e+00, -5.63902557e-01, -1.65101993e+00,
        -5.55944741e-01, -3.41018438e+00, -2.24465466e+00,
        -3.13058877e+00, -2.70089960e+00, -1.88821292e+00,
        -5.54154456e-01, -3.96459913e+00, -2.13485193e+00,
        -3.94577074e+00, -1.62457860e+00, -2.03990698e+00,
        -3.62910867e+00,  6.33734703e-01,  6.48760676e-01,
         6.21744275e-01,  6.34553194e-01,  6.60345614e-01,
         6.60463989e-01,  7.00949013e-01,  7.31556356e-01,
         7.46921539e-01,  7.23420441e-01,  7.16119647e-0

In [ ]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/Colab_Notebooks/RAVDESS_Emotional_speech_audio/speech-emotion-recognition-ravdess-data/Actor_04/03-01-01-01-02-01-04.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction[0]

'calm'